In [18]:
# !pip install spacy
# !pip install yake
# !pip install gensim
# !pip install pyate 
# !pip install rake-nltk
# !pip install summa
# !python -m spacy download en_core_web_sm
# !pip install keybert
# !pip install huggingface_hu==0.10.1

In [1]:
import bibtexparser
from yake import KeywordExtractor
from rake_nltk import Rake
from sklearn.metrics import precision_recall_fscore_support
from yake import KeywordExtractor
from rake_nltk import Rake
from gensim.models.phrases import Phrases, ENGLISH_CONNECTOR_WORDS
from pyate import combo_basic, basic, cvalues
from summa import keywords as summa_keywords
import spacy
import pandas as pd
from keybert import KeyBERT
from nltk.stem import PorterStemmer
from Levenshtein import distance
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from fuzzywuzzy import fuzz
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from nltk.corpus import stopwords

C:\Users\eno\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\eno\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\huggingface_hub\snapshot_download.py:6: FutureWarning: snapshot_download.py has been made private and will no longer be available from version 0.11. Please use `from huggingface_hub import snapshot_download` to import the only public function in this module. Other members of the file may be changed without a deprecation notice.
  warnings.warn(


In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eno\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
#!pip install huggingface-hub==0.10.1

In [7]:
def extract_keywords_from_abstract(abstract):
    
    # Get the English stopwords
    stop_words = set(stopwords.words('english'))
    abstract = ' '.join([word for word in abstract.split() if word.lower() not in stop_words])

    # Initialize Spacy, YAKE, and RAKE keyword extractors
    nlp = spacy.load("en_core_web_lg")
    kw_extractor = KeywordExtractor() # KeywordExtractor(lan="en", n=3, dedupLim=0.6, dedupFunc='seqm', windowsSize=1, top=20, features=None)
    rake_nltk_var = Rake()

    # Extract keywords using Spacy entities
    doc = nlp(abstract)
    spacy_entities = [ent.text for ent in doc.ents]

    # Extract keywords using Spacy noun chunks
    doc = nlp(abstract)
    spacy_noun_chunks = [chunk.text for chunk in doc.noun_chunks]

    # Extract keywords using YAKE
    yake_keywords = kw_extractor.extract_keywords(abstract)
    yake_keywords = [keyword[0] for keyword in yake_keywords]

    # Extract keywords using RAKE
    rake_nltk_var.extract_keywords_from_text(abstract)
    rake_keywords = rake_nltk_var.get_ranked_phrases()

    # Extract keywords using Pyate
    pyate_combo_basic_keywords = combo_basic(abstract).sort_values(ascending=False).index.str.split().str[0].tolist()
    pyate_basic_keywords = basic(abstract).sort_values(ascending=False).index.str.split().str[0].tolist()
    pyate_cvalues_keywords = cvalues(abstract).sort_values(ascending=False).index.str.split().str[0].tolist()

    # Extract keywords using summa
    summa_keywords_ = [keyword[0] for keyword in summa_keywords.keywords(abstract, scores=True)]
    
    # Extract keywords using KeyBERT
    keybert_model = KeyBERT()#KeyBERT(model="m3rg-iitd/matscibert")#KeyBERT()
    keybert_keywords = [keyword[0] for keyword in keybert_model.extract_keywords(abstract, keyphrase_ngram_range=(1, 3), stop_words='english')] #keyphrase_ngram_range=(1, 3),
    
    # Extract keywords using KeyBERT+MatSciBERT
    keybert_m_model = KeyBERT(model="m3rg-iitd/matscibert")#KeyBERT()
    keybert_m_keywords = [keyword[0] for keyword in keybert_m_model.extract_keywords(abstract, keyphrase_ngram_range=(1, 3), stop_words='english')] #keyphrase_ngram_range=(1, 3),

    # Extract keywords using TF-IDF
    tfidf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 3))
    tfidf_matrix = tfidf_vectorizer.fit_transform([abstract])
    tfidf_keywords = tfidf_vectorizer.get_feature_names_out()

    # Extract keywords using LSA
    lsa_model = TruncatedSVD(n_components=10)  # Adjust the number of components as needed
    lsa_matrix = lsa_model.fit_transform(tfidf_matrix)
    lsa_keywords = [tfidf_keywords[i] for i in lsa_model.components_[0].argsort()[::-1]]

    # Extract keywords using LDA
    dictionary = Dictionary([abstract.split()])
    corpus = [dictionary.doc2bow(abstract.split())]
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in [abstract.split()]]
    lda_model = LdaModel(corpus=doc_term_matrix, num_topics=10, id2word=dictionary)  # Adjust the number of topics as needed
    lda_keywords = [word for word, _ in lda_model.show_topic(0)]

    return {
        "Spacy_entities": spacy_entities,
        "Spacy_noun_chunks": spacy_noun_chunks,
        "YAKE_keywords": yake_keywords,
        "RAKE_keywords": rake_keywords,
        "Pyate_combo_basic_keywords": pyate_combo_basic_keywords,
        "Pyate_basic_keywords": pyate_basic_keywords,
        "Pyate_cvalues_keywords": pyate_cvalues_keywords,
        "Summa_keywords": summa_keywords_,
        "Keybert_keywords": keybert_keywords,
        "Keybert_m_keywords": keybert_m_keywords,
        "TFIDF_keywords": tfidf_keywords,
        "LSA_keywords": lsa_keywords,
        "LDA_keywords": lda_keywords
    }


In [8]:
# Example usage
abstract = "Functionalization facilitates targeted delivery of these nanoparticles to various cell types, bioimaging, gene delivery, drug delivery and other therapeutic and diagnostic applications."
keywords = extract_keywords_from_abstract(abstract)
for method, extracted_keywords in keywords.items():
    print(method + ": ", extracted_keywords)

C:\Users\eno\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_lg' (3.0.0) was trained with spaCy v3.0.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
Some weights of the model checkpoint at C:\Users\eno/.cache\torch\sentence_transformers\m3rg-iitd_matscibert were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertMo

Spacy_entities:  ['Functionalization']
Spacy_noun_chunks:  ['Functionalization', 'various cell types']
YAKE_keywords:  ['therapeutic diagnostic applications', 'Functionalization facilitates targeted', 'drug delivery therapeutic', 'facilitates targeted delivery', 'targeted delivery nanoparticles', 'delivery therapeutic diagnostic', 'Functionalization facilitates', 'cell types', 'diagnostic applications', 'facilitates targeted', 'nanoparticles various cell', 'therapeutic diagnostic', 'gene delivery', 'drug delivery', 'targeted delivery', 'delivery nanoparticles', 'delivery therapeutic', 'bioimaging', 'delivery', 'Functionalization']
RAKE_keywords:  ['functionalization facilitates targeted delivery nanoparticles various cell types', 'drug delivery therapeutic diagnostic applications', 'gene delivery', 'bioimaging']
Pyate_combo_basic_keywords:  ['delivery', 'therapeutic', 'drug', 'diagnostic', 'therapeutic', 'delivery', 'delivery', 'drug', 'drug', 'drug', 'various', 'cell', 'various', 'fun

In [9]:
# Function to tokenize and stem text
def tokenize_and_stem(text):
    stemmer = PorterStemmer()
    if isinstance(text, str):
        tokens = [stemmer.stem(word) for word in text.split()]
        return ' '.join(tokens)
    else:
        return str(text)

# Function to calculate Levenshtein distance similarity
def levenshtein_similarity(text1, text2):
    return 1 - (distance(text1, text2) / max(len(text1), len(text2)))

# Function to find synonyms using WordNet
def find_synonyms(word):
    synonyms = set()
    for synset in wordnet.synsets(word):
        for lemma in synset.lemmas():
            synonyms.add(lemma.name())
    return synonyms

# Function to calculate cosine similarity using TF-IDF
def cosine_similarity_score(text1, text2):
    vectorizer = TfidfVectorizer(tokenizer=lambda x: x.split())
    tfidf_matrix = vectorizer.fit_transform([text1, text2])
    return cosine_similarity(tfidf_matrix)[0][1]

# Function to calculate fuzzy matching score
def fuzzy_matching_score(text1, text2):
    return fuzz.token_set_ratio(text1, text2)

# Function to evaluate keywords
def evaluate_keywords(ground_truth_keywords, extracted_keywords):
    # Initialize variables for evaluation metrics
    tp, fp, fn = 0, 0, 0

    # Tokenize and stem ground truth keywords
    ground_truth_stems = [tokenize_and_stem(keyword) for keyword in ground_truth_keywords]

    # Iterate over extracted keywords
    for extracted_keyword in extracted_keywords:
        # Tokenize and stem extracted keyword
        extracted_stem = tokenize_and_stem(extracted_keyword)

        # Check if extracted keyword matches any ground truth keyword
        matched = False
        for ground_truth_stem in ground_truth_stems:
            # Calculate similarity scores
            levenshtein_sim = levenshtein_similarity(extracted_stem, ground_truth_stem)
            cosine_sim = cosine_similarity_score(extracted_stem, ground_truth_stem)
            fuzzy_score = fuzzy_matching_score(extracted_keyword, ground_truth_stem)

            # If any similarity score exceeds threshold, consider it a match
            if levenshtein_sim > 0.8 or cosine_sim > 0.8 or fuzzy_score > 80:
                matched = True
                break

        # Update evaluation metrics based on match status
        if matched:
            tp += 1
        else:
            fp += 1

    # Calculate false negatives (missed ground truth keywords)
    fn = len(ground_truth_keywords) - tp

    # Calculate precision, recall, and F1-score
    if tp + fp > 0:
        precision = tp / (tp + fp)
    else:
        precision = 0.0

    if tp + fn > 0:
        recall = tp / (tp + fn)
    else:
        recall = 0.0

    if precision + recall > 0:
        f1_score = 2 * (precision * recall) / (precision + recall)
    else:
        f1_score = 0.0

    return precision, recall, f1_score

In [11]:
def evaluate_keywords_from_bib(bib_file, extraction_functions):
    # Load the BibTeX file
    with open(bib_file, 'r', encoding='utf-8') as bibfile:
        bib_database = bibtexparser.load(bibfile)

    # Initialize dictionaries to store cumulative scores
    cumulative_precision = {method: 0 for method in extraction_functions}
    cumulative_recall = {method: 0 for method in extraction_functions}
    cumulative_f1_score = {method: 0 for method in extraction_functions}
    total_abstracts = 0

    # Iterate over entries in the BibTeX file
    for entry in bib_database.entries:
        # Check if the entry has abstract and keywords
        if 'abstract' in entry and 'keywords' in entry:
            abstract = entry['abstract'].lower() 
            ground_truth_keywords = entry['keywords'].split(',')
            total_abstracts += 1

            # Evaluate keywords for each extraction function
            for method, extraction_function in extraction_functions.items():
                extracted_keywords = extraction_function(abstract)
                precision, recall, f1_score = evaluate_keywords(ground_truth_keywords, extracted_keywords)

                # Accumulate scores
                cumulative_precision[method] += precision
                cumulative_recall[method] += recall
                cumulative_f1_score[method] += f1_score

    # Calculate averages
    average_precision = {method: cumulative_precision[method] / total_abstracts for method in extraction_functions}
    average_recall = {method: cumulative_recall[method] / total_abstracts for method in extraction_functions}
    average_f1_score = {method: cumulative_f1_score[method] / total_abstracts for method in extraction_functions}

    # Print average scores
    print("Average Scores over all Abstracts:")
    for method in extraction_functions:
        print(f"{method} - Average Precision: {average_precision[method]}, Average Recall: {average_recall[method]}, Average F1-score: {average_f1_score[method]}")


# Define extraction functions
extraction_functions = {
    "Spacy_entities": lambda abstract: extract_keywords_from_abstract(abstract)["Spacy_entities"],
    "Spacy_noun_chunks": lambda abstract: extract_keywords_from_abstract(abstract)["Spacy_noun_chunks"],
    "YAKE_keywords": lambda abstract: extract_keywords_from_abstract(abstract)["YAKE_keywords"],
    "RAKE_keywords": lambda abstract: extract_keywords_from_abstract(abstract)["RAKE_keywords"],
    "Pyate_combo_basic_keywords": lambda abstract: extract_keywords_from_abstract(abstract)["Pyate_combo_basic_keywords"],
    "Pyate_basic_keywords": lambda abstract: extract_keywords_from_abstract(abstract)["Pyate_basic_keywords"],
    "Pyate_cvalues_keywords": lambda abstract: extract_keywords_from_abstract(abstract)["Pyate_cvalues_keywords"],
    "Summa_keywords": lambda abstract: extract_keywords_from_abstract(abstract)["Summa_keywords"],
    "Keybert_keywords": lambda abstract: extract_keywords_from_abstract(abstract)["Keybert_keywords"],
    "Keybert_m_keywords": lambda abstract: extract_keywords_from_abstract(abstract)["Keybert_m_keywords"],
    "TFIDF_keywords": lambda abstract: extract_keywords_from_abstract(abstract)["TFIDF_keywords"],
    "LSA_keywords": lambda abstract: extract_keywords_from_abstract(abstract)["LSA_keywords"],
    "LDA_keywords": lambda abstract: extract_keywords_from_abstract(abstract)["LDA_keywords"],
}

In [12]:
# Example usage
bib_file = "nanomaterials-v01-i01_20240418.bib"  # Replace with the path to your BibTeX file
evaluate_keywords_from_bib(bib_file, extraction_functions)


C:\Users\eno\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_lg' (3.0.0) was trained with spaCy v3.0.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
Some weights of the model checkpoint at C:\Users\eno/.cache\torch\sentence_transformers\m3rg-iitd_matscibert were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertMo

Average Scores over all Abstracts:
Spacy_entities - Average Precision: 0.22857142857142856, Average Recall: 0.02777777777777778, Average F1-score: 0.04731481481481481
Spacy_noun_chunks - Average Precision: 0.31822482357582377, Average Recall: 0.4001625908204856, Average F1-score: 0.3424801960236789
YAKE_keywords - Average Precision: 0.17500000000000002, Average Recall: 0.18398932938406623, Average F1-score: 0.17559951469056598
RAKE_keywords - Average Precision: 0.19403781403781406, Average Recall: 0.18097984861142755, Average F1-score: 0.18194505829288435
Pyate_combo_basic_keywords - Average Precision: 0.20507481341586767, Average Recall: 0.8979371408318777, Average F1-score: 0.32837471921654104
Pyate_basic_keywords - Average Precision: 0.20507481341586767, Average Recall: 0.8979371408318777, Average F1-score: 0.32837471921654104
Pyate_cvalues_keywords - Average Precision: 0.20507481341586767, Average Recall: 0.8979371408318777, Average F1-score: 0.32837471921654104
Summa_keywords - Av